# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

In [1]:
import torch

In [2]:
import argparse

In [3]:
from __future__ import print_function
import torch


In [4]:
import torchvision

In [5]:
import torchvision.transforms as transforms

In [6]:
from __future__ import print_function, division
import os
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from PIL import Image

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [7]:
data_dir = 'flowers' 
train_dir = data_dir + '/train' 
valid_dir = data_dir + '/valid' 
test_dir = data_dir + '/test'


In [8]:
# TODO: Define your transforms for the training, validation, and testing sets

training_transforms = transforms.Compose([transforms.RandomRotation(30),
                                          transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

training_dataset = torchvision.datasets.ImageFolder(train_dir, transform=training_transforms)

valid_transforms = transforms.Compose([transforms.RandomRotation(30),
                                          transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

validset = torchvision.datasets.ImageFolder(valid_dir, transform=valid_transforms)


test_transforms = transforms.Compose([transforms.RandomRotation(30),
                                          transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

testset = torchvision.datasets.ImageFolder(test_dir, transform=test_transforms)



# TODO: Load the datasets with ImageFolder

# TODO: Using the image datasets and the trainforms, define the dataloaders

train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64,
                                          shuffle=True)
valloader = torch.utils.data.DataLoader(validset, batch_size=64,
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False)


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [9]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [10]:
# TODO: Build and train your network

In [11]:
# Hyperparameters for our network 
input_size = 25088
hidden_sizes = [5024, 1024, 256]
output_size = 102
dropout = 0.4

In [12]:
#import models from torchvision
from torchvision import models
#build the pretrained model (vgg16 in this case)
# changed to vgg19 (Dec 23 2018)

model = models.vgg16(pretrained = True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:05<00:00, 98457770.76it/s] 


In [13]:
#Fraeze parameters so we no backprop and please delete if this does not work
for param in model.parameters():
    param.requires_grad = False
    

In [14]:
#import OrderedDicted to corectly align the network layers
#import nn you use activation and dropout features
from collections import OrderedDict
from torch import nn

In [15]:
#create classifier
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 5024)),
                           ('relu1', nn.ReLU()), 
                           ('dropout1', nn.Dropout(p=0.337)),
                           ('fc2', nn.Linear(5024, 1024)),
                           ('relu2', nn.ReLU()),
                           ('dropout2', nn.Dropout(p=0.337)),           
                           ('fc3', nn.Linear(1024, 256)),
                           ('relu3', nn.ReLU()),
                           ('dropout3', nn.Dropout(dropout)),
                           ('output', nn.Linear(256, 102)),
                           ('softmax', nn.LogSoftmax(dim=1))
 ]))

In [16]:
#replace the model's classifier with this new classifier 

model.classifier = classifier
model.classifier

Sequential(
  (fc1): Linear(in_features=25088, out_features=5024, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.337)
  (fc2): Linear(in_features=5024, out_features=1024, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.337)
  (fc3): Linear(in_features=1024, out_features=256, bias=True)
  (relu3): ReLU()
  (dropout3): Dropout(p=0.4)
  (output): Linear(in_features=256, out_features=102, bias=True)
  (softmax): LogSoftmax()
)

In [17]:
#import optimizer for 
from torch import optim

In [18]:
import time

In [19]:
#define criteria and optimizer
for device in ['cpu','cuda']:
    
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr = 0.005)
    
    model.to(device)
    

In [20]:
#validation
def validation(model, valloader, criterion):
    validation_loss = 0
    accuracy = 0
    
    for images, labels in valloader:
        images, labels = images.to(device), labels.to(device)
        
        output = model.forward(images)
        validation_loss = criterion(output, labels).item()
       
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return validation_loss, accuracy
    

In [21]:
# running validation

epochs = 5
steps = 0
running_loss = 0
print_every = 10
print('training started')

for e in range(epochs):
        model.to(device)
        for images, labels in train_loader:
            steps += 1
            images, labels = images.to(device), labels.to(device)


            optimizer.zero_grad()

            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                # Make sure network is in eval mode for inference
                model.eval()

                # Turn off gradients for validation, saves memory and computations
                with torch.no_grad():
                    validation_loss, accuracy = validation(model, valloader, criterion)

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Valid Loss: {:.3f}.. ".format(validation_loss/len(valloader)),
                      "Validation Accuracy: {:.3f}".format(accuracy/len(valloader)))

                running_loss = 0

                # Make sure training is back on
            model.train()
print('training  model completed')

training started
Epoch: 1/5..  Training Loss: 12.479..  Valid Loss: 0.358..  Validation Accuracy: 0.036
Epoch: 1/5..  Training Loss: 4.653..  Valid Loss: 0.344..  Validation Accuracy: 0.045
Epoch: 1/5..  Training Loss: 4.549..  Valid Loss: 0.333..  Validation Accuracy: 0.065
Epoch: 1/5..  Training Loss: 4.412..  Valid Loss: 0.310..  Validation Accuracy: 0.105
Epoch: 1/5..  Training Loss: 4.362..  Valid Loss: 0.322..  Validation Accuracy: 0.100
Epoch: 1/5..  Training Loss: 4.325..  Valid Loss: 0.324..  Validation Accuracy: 0.096
Epoch: 1/5..  Training Loss: 4.309..  Valid Loss: 0.342..  Validation Accuracy: 0.094
Epoch: 1/5..  Training Loss: 4.289..  Valid Loss: 0.315..  Validation Accuracy: 0.116
Epoch: 1/5..  Training Loss: 4.271..  Valid Loss: 0.310..  Validation Accuracy: 0.106
Epoch: 1/5..  Training Loss: 4.122..  Valid Loss: 0.288..  Validation Accuracy: 0.114
Epoch: 2/5..  Training Loss: 4.077..  Valid Loss: 0.297..  Validation Accuracy: 0.106
Epoch: 2/5..  Training Loss: 4.115..

In [22]:
# TODO: Do validation on the test set

with torch.no_grad():
    test_loss, accuracy = validation(model, testloader, criterion)

print("Test-Loss: {}\n".format(test_loss/len(testloader)),
     "Test-Accuracy: {}".format(accuracy/len(testloader)))

Test-Loss: 0.024026114766190692
 Test-Accuracy: 0.08780487626791


In [23]:
# TODO: Save the checkpoint 
checkpoint = {
    'arch': 'vgg16',
    'input_size': model.classifier[0].in_features,
    'state_dict': model.classifier.state_dict(),
    'class_to_idx': training_dataset.class_to_idx,
    'output_size': output_size,
    'classifier': model.classifier,
    'hidden_layers': hidden_sizes,
    'dropout': dropout,
    'optimizer state': optimizer.state_dict,
    'number of epochs': epochs,
}

torch.save(checkpoint, 'checkpoint-jupyter.pth')
print('saved checkpoint')

saved checkpoint


In [24]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    #arch = checkpoint['arch']
    model = models.vgg16(pretrained=True)
    classifier = checkpoint['classifier']
    model.classifier = classifier
    model.classifier.load_state_dict(checkpoint['state_dict'])
    
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model, checkpoint['class_to_idx']
    
model, class_to_idx = load_checkpoint('checkpoint-jupyter.pth')
print('model is loaded')

model is loaded


In [25]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # Open the image and resize it
    image_pil = Image.open(image)
    image_pil = image_pil.resize((256,256))
    
    # Resize 
    width, height = image_pil.size   # Get dimensions

    left = (width - 224)/2
    top = (height - 224)/2
    right = (width + 224)/2
    bottom = (height + 224)/2

    image_pil = image_pil.crop((left, top, right, bottom))
    
    # Turn into a np_array
    np_image = np.array(image_pil)/255
    
    # Undo mean and std and then transpose the array
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])  
    np_image = (np_image - mean)/std
    np_image = np.transpose(np_image, (2, 0, 1))
    
    return np_image

In [26]:
# Test process_image implementation with a random picture from the test set
test = process_image('flowers/test/11/image_03115.jpg')
print(test.shape)

(3, 224, 224)


In [27]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [28]:
imshow(test)

In [29]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Inverting class_to_idx 
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    
    model= model.to(device)
    img_predict = torch.FloatTensor([process_image(image_path)])
    img_predict = img_predict.to(device)

    with torch.no_grad():
        result = model(img_predict)
    
    
    # Save probs and classes
    probs = np.array(torch.exp(result).topk(topk)[0])
    classes = np.array(torch.exp(result).topk(topk)[1])
    classes = [idx_to_class[c] for c in classes[0,:]]  
    
    return(probs, classes)

In [30]:
#testing it
print(predict('flowers/test/11/image_03115.jpg', model))

(array([[ 0.03500159,  0.02441889,  0.02382321,  0.01947578,  0.01880944]], dtype=float32), ['51', '74', '83', '95', '89'])


In [31]:
# TODO: Display an image along with the top 5 classes
def sanity_check(pathToImage, model):
    #process image
    processed_image = process_image(pathToImage)
    #get the probability and the top classes for the image
    probs, classes = predict(pathToImage, model)
    #create subplot
    fig, (p1,p2) = plt.subplots(figsize=(10,10), ncols=1, nrows=2)                         
    #print(cat_to_name[classes[0]])
    p1.set_title(cat_to_name[classes[0]])                          
    # Show the image for reference
    p1.imshow(Image.open(pathToImage))
    p1.axis('off')
                  
    y_pos = np.arange(len(probs[0]))

    p2.barh(y_pos, probs[0])
    plt.yticks(y_pos)
    p2.set_yticklabels([cat_to_name[c] for c in classes])
    p2.invert_yaxis()
    p2.set_title("Class Probabilities")

In [32]:
sanity_check('flowers/test/85/image_04805.jpg', model)

In [ ]:
#define criteria and optimizer

    
    for ii, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        start = time.time()
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if ii==3:
            break
            
    print(f"DEVICE ={device}; Time per batch: {(time.time() - start)/3:.3f} seconds")   
    

In [ ]:
# validation function

epochs = 3
print_every = 40
steps = 0

# change to cuda
model.to('cuda')

for e in range(epochs):
    running_loss = 0
    for ii, (inputs, labels) in enumerate(train_loader):
        steps += 1
        
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        
        # Forward and backward passes
        
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            print("Epoch: {}/{}...".format(e+1, epochs),
                 "Loss: {:.4f}".format(running_loss/print_every))
            
            running_loss = 0
            
        


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        ouptuts = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy of the network on the test images: %d %%' % (100 * correct/total))

In [ ]:
# introduced copy on Dec 21 2018
import copy

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes